In [3]:
!pip install catboost lightgbm

In [45]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
test = pd.read_csv("test.csv")
df = pd.read_csv("train.csv")
df_new = df.drop("target", axis = 1)

In [46]:
df

,Month,DayofMonth,DayOfWeek,DepTime,ArrTime,UniqueCarrier,FlightNum,TailNum,AirTime,Origin,Dest,Distance,TaxiIn,TaxiOut,target,Id
0,8,14,4,1334.0,1440.0,XE,430,N13553,54.0,FAT,SAN,314,3.0,9.0,0,FYNC08ZRT1MS
1,5,3,6,1706.0,2016.0,XE,2567,N12924,103.0,BNA,EWR,748,9.0,18.0,1,4M3EPJ0BFKNV
2,7,11,5,1323.0,1507.0,XE,2989,N19966,125.0,CLT,IAH,913,6.0,33.0,1,OR30D5LJS4AF
3,12,11,4,2039.0,2238.0,WN,3743,N215WN,224.0,BWI,DEN,1491,5.0,10.0,1,0PRTFL3QD5ZI
4,5,23,5,1628.0,1744.0,UA,926,N347UA,60.0,LAX,SFO,337,3.0,13.0,1,DR7UZO31NC2A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1999995,11,22,6,1119.0,1238.0,AA,2421,N383AA,174.0,DFW,LAX,1235,10.0,15.0,0,KHV6OJ08LE0S
1999996,2,25,1,759.0,858.0,AA,1109,N431AA,36.0,DFW,AUS,190,6.0,17.0,0,WM56NFH3B82U
1999997,9,23,2,1226.0,2109.0,DL,162,N3763D,293.0,SEA,JFK,2421,30.0,20.0,0,K6UIWFZ0R1T7
1999998,7,26,6,729.0,929.0,US,1099,N956UW,97.0,PHL,JAX,742,3.0,20.0,0,RMFGZOY59UCK


In [47]:
columns = ["Month", "DayofMonth", "DayOfWeek", "UniqueCarrier", "Origin", "Dest"]

In [48]:
CatBoost = CatBoostClassifier(cat_features = columns, depth = 8)

In [49]:
target = df["target"]

In [50]:
df_new = df_new.fillna(0)

In [51]:
origin_mte = df[['Origin', 'target']].groupby('Origin')['target'].mean()
origin_mte.name = 'Origin_MTE'
df_new = pd.merge(df_new, origin_mte, how='left', on='Origin')

In [52]:
Dest_mte = df[['Dest', 'target']].groupby('Dest')['target'].mean()
Dest_mte.name = 'Dest_MTE'
df_new = pd.merge(df_new, Dest_mte, how='left', on='Dest')

In [53]:
df_new = df_new.drop("Id", axis = 1)
df_new = df_new.drop("TailNum", axis = 1)

In [54]:
smth = df['TaxiIn'] + df['TaxiOut'] +  df['AirTime']
df.insert(loc=len(df.columns), column='Smth', value=smth) 

In [55]:
df["DepTime"] = df["DepTime"]/100
df["ArrTime"] = df["ArrTime"]/100

In [56]:
df_new.head()

,Month,DayofMonth,DayOfWeek,DepTime,ArrTime,UniqueCarrier,FlightNum,AirTime,Origin,Dest,Distance,TaxiIn,TaxiOut,Origin_MTE,Dest_MTE
0,8,14,4,1334.0,1440.0,XE,430,54.0,FAT,SAN,314,3.0,9.0,0.203920,0.301685
1,5,3,6,1706.0,2016.0,XE,2567,103.0,BNA,EWR,748,9.0,18.0,0.302108,0.418648
2,7,11,5,1323.0,1507.0,XE,2989,125.0,CLT,IAH,913,6.0,33.0,0.297510,0.250599
3,12,11,4,2039.0,2238.0,WN,3743,224.0,BWI,DEN,1491,5.0,10.0,0.314572,0.277956
4,5,23,5,1628.0,1744.0,UA,926,60.0,LAX,SFO,337,3.0,13.0,0.289365,0.344834


In [57]:
Cat = CatBoost.fit(df_new, target)

Learning rate set to 0.264529
0:	learn: 0.6218822	total: 2.62s	remaining: 43m 37s
1:	learn: 0.5790288	total: 5.13s	remaining: 42m 40s
2:	learn: 0.5547740	total: 8.03s	remaining: 44m 29s
3:	learn: 0.5441960	total: 10.6s	remaining: 43m 53s
4:	learn: 0.5350155	total: 13.3s	remaining: 44m 12s
5:	learn: 0.5267020	total: 16.1s	remaining: 44m 22s
6:	learn: 0.5232482	total: 18.4s	remaining: 43m 24s
7:	learn: 0.5207328	total: 20.7s	remaining: 42m 52s
8:	learn: 0.5173935	total: 23.5s	remaining: 43m 9s
9:	learn: 0.5144051	total: 25.9s	remaining: 42m 39s
10:	learn: 0.5127018	total: 28.2s	remaining: 42m 19s
11:	learn: 0.5085873	total: 31s	remaining: 42m 34s
12:	learn: 0.5066499	total: 33.9s	remaining: 42m 50s
13:	learn: 0.5051288	total: 36.2s	remaining: 42m 32s
14:	learn: 0.5030209	total: 38.4s	remaining: 42m 3s
15:	learn: 0.5016603	total: 40s	remaining: 41m 1s
16:	learn: 0.5008519	total: 42.5s	remaining: 40m 59s
17:	learn: 0.4999964	total: 44.9s	remaining: 40m 50s
18:	learn: 0.4993348	total: 47.5s

153:	learn: 0.4433520	total: 6m 54s	remaining: 37m 57s
154:	learn: 0.4424951	total: 6m 57s	remaining: 37m 55s
155:	learn: 0.4417163	total: 6m 59s	remaining: 37m 50s
156:	learn: 0.4416362	total: 7m 2s	remaining: 37m 48s
157:	learn: 0.4407697	total: 7m 4s	remaining: 37m 43s
158:	learn: 0.4404357	total: 7m 7s	remaining: 37m 40s
159:	learn: 0.4403471	total: 7m 10s	remaining: 37m 37s
160:	learn: 0.4398796	total: 7m 12s	remaining: 37m 33s
161:	learn: 0.4397805	total: 7m 15s	remaining: 37m 30s
162:	learn: 0.4397156	total: 7m 17s	remaining: 37m 26s
163:	learn: 0.4391848	total: 7m 20s	remaining: 37m 26s
164:	learn: 0.4384516	total: 7m 23s	remaining: 37m 24s
165:	learn: 0.4383625	total: 7m 25s	remaining: 37m 20s
166:	learn: 0.4380744	total: 7m 28s	remaining: 37m 17s
167:	learn: 0.4371847	total: 7m 31s	remaining: 37m 13s
168:	learn: 0.4362730	total: 7m 33s	remaining: 37m 11s
169:	learn: 0.4358513	total: 7m 36s	remaining: 37m 9s
170:	learn: 0.4357727	total: 7m 39s	remaining: 37m 6s
171:	learn: 0.4

302:	learn: 0.4008243	total: 13m 39s	remaining: 31m 24s
303:	learn: 0.4007427	total: 13m 42s	remaining: 31m 22s
304:	learn: 0.3998082	total: 13m 44s	remaining: 31m 19s
305:	learn: 0.3997643	total: 13m 47s	remaining: 31m 17s
306:	learn: 0.3995494	total: 13m 50s	remaining: 31m 14s
307:	learn: 0.3995335	total: 13m 53s	remaining: 31m 11s
308:	learn: 0.3994842	total: 13m 55s	remaining: 31m 8s
309:	learn: 0.3993054	total: 13m 58s	remaining: 31m 5s
310:	learn: 0.3992685	total: 14m	remaining: 31m 2s
311:	learn: 0.3992228	total: 14m 3s	remaining: 30m 59s
312:	learn: 0.3991329	total: 14m 6s	remaining: 30m 57s
313:	learn: 0.3987935	total: 14m 9s	remaining: 30m 54s
314:	learn: 0.3987103	total: 14m 11s	remaining: 30m 52s
315:	learn: 0.3986714	total: 14m 14s	remaining: 30m 50s
316:	learn: 0.3985852	total: 14m 17s	remaining: 30m 48s
317:	learn: 0.3983813	total: 14m 20s	remaining: 30m 46s
318:	learn: 0.3983341	total: 14m 23s	remaining: 30m 44s
319:	learn: 0.3982952	total: 14m 26s	remaining: 30m 41s
32

450:	learn: 0.3803369	total: 20m 27s	remaining: 24m 54s
451:	learn: 0.3803124	total: 20m 30s	remaining: 24m 51s
452:	learn: 0.3802729	total: 20m 33s	remaining: 24m 49s
453:	learn: 0.3800694	total: 20m 36s	remaining: 24m 47s
454:	learn: 0.3800120	total: 20m 39s	remaining: 24m 44s
455:	learn: 0.3799645	total: 20m 42s	remaining: 24m 42s
456:	learn: 0.3799510	total: 20m 45s	remaining: 24m 39s
457:	learn: 0.3799381	total: 20m 47s	remaining: 24m 36s
458:	learn: 0.3799235	total: 20m 50s	remaining: 24m 33s
459:	learn: 0.3793444	total: 20m 52s	remaining: 24m 30s
460:	learn: 0.3792990	total: 20m 55s	remaining: 24m 28s
461:	learn: 0.3792661	total: 20m 58s	remaining: 24m 25s
462:	learn: 0.3792580	total: 21m 1s	remaining: 24m 22s
463:	learn: 0.3792398	total: 21m 4s	remaining: 24m 20s
464:	learn: 0.3788567	total: 21m 6s	remaining: 24m 17s
465:	learn: 0.3788104	total: 21m 9s	remaining: 24m 14s
466:	learn: 0.3781319	total: 21m 12s	remaining: 24m 12s
467:	learn: 0.3780922	total: 21m 15s	remaining: 24m 

598:	learn: 0.3652235	total: 27m 6s	remaining: 18m 9s
599:	learn: 0.3652009	total: 27m 10s	remaining: 18m 6s
600:	learn: 0.3651665	total: 27m 12s	remaining: 18m 3s
601:	learn: 0.3651424	total: 27m 15s	remaining: 18m
602:	learn: 0.3650467	total: 27m 18s	remaining: 17m 58s
603:	learn: 0.3650288	total: 27m 20s	remaining: 17m 55s
604:	learn: 0.3649767	total: 27m 23s	remaining: 17m 53s
605:	learn: 0.3648188	total: 27m 26s	remaining: 17m 50s
606:	learn: 0.3647926	total: 27m 29s	remaining: 17m 47s
607:	learn: 0.3647519	total: 27m 31s	remaining: 17m 44s
608:	learn: 0.3647270	total: 27m 34s	remaining: 17m 42s
609:	learn: 0.3647063	total: 27m 36s	remaining: 17m 39s
610:	learn: 0.3646866	total: 27m 40s	remaining: 17m 37s
611:	learn: 0.3646736	total: 27m 43s	remaining: 17m 34s
612:	learn: 0.3646375	total: 27m 46s	remaining: 17m 31s
613:	learn: 0.3645633	total: 27m 48s	remaining: 17m 28s
614:	learn: 0.3645339	total: 27m 50s	remaining: 17m 25s
615:	learn: 0.3645005	total: 27m 53s	remaining: 17m 23s


746:	learn: 0.3518821	total: 33m 54s	remaining: 11m 29s
747:	learn: 0.3517226	total: 33m 57s	remaining: 11m 26s
748:	learn: 0.3516924	total: 33m 59s	remaining: 11m 23s
749:	learn: 0.3516700	total: 34m 2s	remaining: 11m 20s
750:	learn: 0.3516398	total: 34m 5s	remaining: 11m 18s
751:	learn: 0.3516087	total: 34m 7s	remaining: 11m 15s
752:	learn: 0.3515999	total: 34m 10s	remaining: 11m 12s
753:	learn: 0.3513279	total: 34m 13s	remaining: 11m 9s
754:	learn: 0.3512926	total: 34m 16s	remaining: 11m 7s
755:	learn: 0.3511909	total: 34m 19s	remaining: 11m 4s
756:	learn: 0.3510209	total: 34m 21s	remaining: 11m 1s
757:	learn: 0.3510044	total: 34m 24s	remaining: 10m 59s
758:	learn: 0.3509717	total: 34m 27s	remaining: 10m 56s
759:	learn: 0.3509509	total: 34m 29s	remaining: 10m 53s
760:	learn: 0.3509398	total: 34m 32s	remaining: 10m 50s
761:	learn: 0.3509229	total: 34m 35s	remaining: 10m 48s
762:	learn: 0.3506907	total: 34m 37s	remaining: 10m 45s
763:	learn: 0.3505830	total: 34m 40s	remaining: 10m 42s

896:	learn: 0.3405052	total: 40m 41s	remaining: 4m 40s
897:	learn: 0.3404868	total: 40m 44s	remaining: 4m 37s
898:	learn: 0.3404683	total: 40m 46s	remaining: 4m 34s
899:	learn: 0.3404572	total: 40m 49s	remaining: 4m 32s
900:	learn: 0.3404486	total: 40m 51s	remaining: 4m 29s
901:	learn: 0.3403528	total: 40m 54s	remaining: 4m 26s
902:	learn: 0.3403383	total: 40m 57s	remaining: 4m 23s
903:	learn: 0.3403261	total: 41m	remaining: 4m 21s
904:	learn: 0.3403019	total: 41m 2s	remaining: 4m 18s
905:	learn: 0.3402724	total: 41m 5s	remaining: 4m 15s
906:	learn: 0.3400876	total: 41m 8s	remaining: 4m 13s
907:	learn: 0.3400745	total: 41m 10s	remaining: 4m 10s
908:	learn: 0.3397943	total: 41m 13s	remaining: 4m 7s
909:	learn: 0.3397275	total: 41m 16s	remaining: 4m 4s
910:	learn: 0.3396241	total: 41m 19s	remaining: 4m 2s
911:	learn: 0.3395438	total: 41m 21s	remaining: 3m 59s
912:	learn: 0.3395088	total: 41m 24s	remaining: 3m 56s
913:	learn: 0.3394748	total: 41m 26s	remaining: 3m 53s
914:	learn: 0.339433

In [58]:
test = test.fillna(0)

In [59]:
test.head()

,Month,DayofMonth,DayOfWeek,DepTime,ArrTime,UniqueCarrier,FlightNum,TailNum,AirTime,Origin,Dest,Distance,TaxiIn,TaxiOut,Id
0,8,15,5,806.0,957.0,UA,1157,N834UA,111.0,DFW,DEN,641,8.0,52.0,QKXNUJO50EPF
1,10,14,2,2020.0,2133.0,WN,1965,N203WN,49.0,PHX,ONT,325,4.0,20.0,Y2XFDOHQK6VM
2,6,1,7,2245.0,9.0,FL,910,N943AT,67.0,ATL,PHF,508,6.0,11.0,TU5QMACJHR10
3,12,31,3,1638.0,1753.0,OO,5732,N567SW,66.0,GEG,SEA,224,3.0,6.0,0731ET5Z9FWJ
4,10,23,4,1544.0,2358.0,AA,22,N329AA,289.0,LAX,JFK,2475,12.0,13.0,5BNGICLQFKSO


In [60]:
test = test.drop("TailNum", axis = 1)

In [61]:
df["DepTime"] = df["DepTime"]/100
df["ArrTime"] = df["ArrTime"]/100

In [62]:
test = test.drop("Id", axis = 1)

In [63]:
df["Smth2"] = df['TaxiIn'] + df['TaxiOut'] +  df['AirTime']

In [64]:
test["Origin_MTE"] = df_new["Origin_MTE"]
test["Dest_MTE"] = df_new["Dest_MTE"]

In [65]:
proba = CatBoost.predict_proba(test)[:, 1]

In [66]:
submission = pd.read_csv("sample_submission.csv")

In [67]:
submission['target'] = proba

In [68]:
submission.to_csv("submission", index = False)